In [167]:
import pandas as pd
import json

lucene_lib = "lucene-9.8.0"
lucene_lib_label = f"{lucene_lib}(μs)"

tantivy_lib = "tantivy-0.21"
tantivy_lib_label = f"{tantivy_lib}(μs)"

results_location = "../web/public/results.json"

In [168]:
def get_mean_per_query(test_type, library):
    results=json.load(open(results_location, "r"))
    df = results[test_type][library]
    df = pd.json_normalize(df)
    df = df.drop("tags", axis=1)
    df1 = pd.concat([pd.DataFrame(df["duration"].values.tolist())], 
             axis=1, 
             keys=df.columns)
    df1.columns = [f'{i}{j}' for i, j in df1.columns]
    df[f"meanDuration"] = df1.mean(axis=1)
    df = df.drop(["duration","count"], axis=1)
    df["library"] = library
    return df


In [196]:
def get_mean_per_query_compare(test_type, library):
    results=json.load(open(results_location, "r"))
    df = results[test_type][library]
    df = pd.json_normalize(df)
    df = df.drop("tags", axis=1)
    df1 = pd.concat([pd.DataFrame(df["duration"].values.tolist())], 
             axis=1, 
             keys=df.columns)
    df1.columns = [f'{i}{j}' for i, j in df1.columns]
    df[f"{library}"] = df1.mean(axis=1)
    df = df.drop(["duration","count"], axis=1)
    return df

In [212]:
lucene_df = get_mean_per_query_compare("TOP_100", "lucene-9.8.0")
tantivy_df = get_mean_per_query_compare("TOP_100", "tantivy-0.21")
df = lucene_df.merge(tantivy_df, on="query")
df

,query,lucene-9.8.0NS,tantivy-0.21NS
0,ref,1021.0,324.1
1,http,120.5,72.8
2,from,4283.0,1343.8
3,title,2693.3,652.2
4,name,2271.8,625.3
...,...,...,...
6160,"""editor first""~4",25887.1,13108.6
6161,"""same name""",42404.2,34338.3
6162,"""same name""~4",49223.8,34664.0
6163,"""larger than""",13265.7,13003.9


In [213]:
df["diff"] = df["lucene-9.8.0(μs)"] - df["tantivy-0.21(μs)"]

In [218]:
df.describe()

,lucene-9.8.0NS,tantivy-0.21NS,diff
count,6165.000000,6165.000000,6165.000000
mean,13384.810251,9395.468500,3989.341752
std,25734.367381,23432.986015,9938.422131
min,12.400000,5.100000,-200532.400000
25%,1209.500000,411.700000,517.700000
50%,3415.300000,1390.300000,1645.200000
75%,15169.200000,8401.200000,5274.700000
max,364744.800000,318468.500000,175670.700000


In [219]:
df[df["diff"]<0].describe()

,lucene-9.8.0NS,tantivy-0.21NS,diff
count,345.00000,345.000000,345.000000
mean,29102.73971,38465.865217,-9363.125507
std,48237.04371,57632.744830,20588.715783
min,14.00000,14.500000,-200532.400000
25%,56.10000,115.900000,-9640.600000
50%,4015.80000,11777.100000,-1726.200000
75%,36562.90000,48145.700000,-66.600000
max,239610.40000,314905.800000,-0.500000
